In [8]:
import numpy as np
from stabalizer_circuit import StabilizerCircuit

# Circuit Functions

Define two helper functions: one to build a 2-qubit Bell pair, and one to build an n-qubit GHZ state.


In [9]:
# Define helper functions
def make_bell() -> StabilizerCircuit:
    sc = StabilizerCircuit(2)
    sc.h(0)
    sc.cx(0, 1)
    return sc

def make_ghz(n: int = 3) -> StabilizerCircuit:
    sc = StabilizerCircuit(n)
    sc.h(0)
    for i in range(1, n):
        sc.cx(0, i)
    return sc

# for printing
def format_complex(arr, tol=1e-8):
    """
    Format a complex scalar or numpy array with 4 decimal places,
    dropping negligible real or imaginary parts.

    Args:
      arr: A complex number or array-like of complex numbers.
      tol: Threshold below which a real/imag part is considered zero.

    Returns:
      If arr is scalar: a formatted string.
      If arr is array-like: prints each row with formatted entries.
    """
    def _fmt(z):
        if abs(z.imag) < tol:
            return f"{z.real:.4f}"
        if abs(z.real) < tol:
            return f"{z.imag:.4f}j"
        return f"{z.real:.4f}{z.imag:+.4f}j"

    arr_np = np.array(arr)
    if arr_np.ndim == 0:  # scalar
        return _fmt(arr_np.item())
    elif arr_np.ndim == 1:
        print("[", "  ".join(_fmt(z) for z in arr_np), "]")
    else:
        for row in arr_np:
            print("[", "  ".join(_fmt(z) for z in row), "]")


# Bell-Pair Analysis

1. Build and display the Bell-pair circuit.  
2. Compute and print the exact statevector.  
3. Perform full Pauli tomography on qubits [0,1] and print the reconstructed density matrix.  
4. Compare to the ideal density matrix and compute the tomography fidelity.


In [10]:
# Bell-pair example

# build the circuit
sc = make_bell()

# print the Stim circuit directly
print("Circuit:")
print(sc.circuit)

# exact statevector
psi = sc.state_vector()
print("\nStatevector:")
format_complex(psi)         

# tomography
rho = sc.tomography_dm([0, 1], shots=2000)
print("\nReconstructed rho:")
format_complex(rho)      

# ideal density and fidelity
rho_ideal = np.outer(psi, psi.conj())
print("\nIdeal rho:")
format_complex(rho_ideal)

fidelity = np.real(psi.conj() @ (rho @ psi))
print("\nFidelity:", f"{fidelity:.4f}")


Circuit:
H 0
CX 0 1

Statevector:
[ 0.7071  0.0000  0.0000  0.7071 ]

Reconstructed rho:
[ 0.4985  -0.0088-0.0163j  -0.0140+0.0010j  0.5000-0.0135j ]
[ -0.0088+0.0163j  -0.0015  -0.0160j  0.0080+0.0010j ]
[ -0.0140-0.0010j  0.0160j  0.0015  0.0088-0.0057j ]
[ 0.5000+0.0135j  0.0080-0.0010j  0.0088+0.0057j  0.5015 ]

Ideal rho:
[ 0.5000  0.0000  0.0000  0.5000 ]
[ 0.0000  0.0000  0.0000  0.0000 ]
[ 0.0000  0.0000  0.0000  0.0000 ]
[ 0.5000  0.0000  0.0000  0.5000 ]

Fidelity: 1.0000


# 3-Qubit GHZ Analysis

1. Build and display the 3-qubit GHZ circuit.  
2. Compute and print the exact statevector.  
3. Perform full Pauli tomography on all three qubits and print the reconstructed density matrix.  
4. Compare to the ideal density matrix and compute the tomography fidelity.


In [11]:
# 3-qubit GHZ example

sc = make_ghz(3)

print("Circuit:")
print(sc.circuit)

psi = sc.state_vector()
print("\nStatevector:")
format_complex(psi)

rho = sc.tomography_dm([0, 1, 2], shots=2000)
print("\nReconstructed rho:")
format_complex(rho)

rho_ideal = np.outer(psi, psi.conj())
print("\nIdeal rho:")
format_complex(rho_ideal)

fidelity = np.real(psi.conj() @ (rho @ psi))
print("\nFidelity:", f"{fidelity:.4f}")


Circuit:
H 0
CX 0 1 0 2

Statevector:
[ 0.7071  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.7071 ]

Reconstructed rho:
[ 0.4986  0.0046-0.0019j  0.0116-0.0069j  0.0020-0.0077j  0.0013+0.0140j  -0.0063-0.0086j  0.0089+0.0037j  0.5000+0.0027j ]
[ 0.0046+0.0019j  -0.0084  0.0057-0.0085j  0.0019+0.0066j  0.0002+0.0066j  -0.0035+0.0025j  -0.0015j  0.0044-0.0050j ]
[ 0.0116+0.0069j  0.0057+0.0085j  -0.0031  0.0099-0.0024j  -0.0019+0.0028j  0.0065j  0.0030-0.0110j  -0.0032+0.0084j ]
[ 0.0020+0.0077j  0.0019-0.0066j  0.0099+0.0024j  -0.0046  -0.0077j  -0.0019+0.0030j  0.0052+0.0006j  0.0037+0.0110j ]
[ 0.0013-0.0140j  0.0002-0.0066j  -0.0019-0.0028j  0.0077j  0.0046  0.0069+0.0021j  0.0079+0.0044j  -0.0170+0.0077j ]
[ -0.0063+0.0086j  -0.0035-0.0025j  -0.0065j  -0.0019-0.0030j  0.0069-0.0021j  0.0031  -0.0068-0.0020j  -0.0149-0.0051j ]
[ 0.0089-0.0037j  0.0015j  0.0030+0.0110j  0.0052-0.0006j  0.0079-0.0044j  -0.0068+0.0020j  0.0084  -0.0054-0.0004j ]
[ 0.5000-0.0027j  0.0044+0.0050j  -0

## Pauli Channel Noise

Apply a single-qubit Pauli error channel on qubit 0 of a Bell pair using `PAULI_CHANNEL_1`.


In [12]:
sc = make_bell()
sc.pauli_channel(0, (0.10, 0.05, 0.02))  # pX=0.10, pY=0.05, pZ=0.02

print("Circuit:")
print(sc.circuit)

psi = sc.state_vector()
print("\nStatevector after Pauli channel:")
format_complex(psi)

rho = sc.tomography_dm([0, 1], shots=2000)
print("\nReconstructed rho:")
format_complex(rho)

rho_ideal = np.outer(psi, psi.conj())
print("\nIdeal rho:")
format_complex(rho_ideal)

fidelity = np.real(psi.conj() @ (rho @ psi))
print("\nFidelity:", f"{fidelity:.4f}")


Circuit:
H 0
CX 0 1
PAULI_CHANNEL_1(0.1, 0.05, 0.02) 0

Statevector after Pauli channel:
[ 0.7071  0.0000  0.0000  0.7071 ]

Reconstructed ρ:
[ 0.4305  -0.0057-0.0052j  -0.0185+0.0145j  0.4057+0.0165j ]
[ -0.0057+0.0052j  0.0635  0.0207+0.0020j  -0.0225-0.0095j ]
[ -0.0185-0.0145j  0.0207-0.0020j  0.0825  0.0063-0.0012j ]
[ 0.4057-0.0165j  -0.0225+0.0095j  0.0063+0.0012j  0.4235 ]

Ideal ρ:
[ 0.5000  0.0000  0.0000  0.5000 ]
[ 0.0000  0.0000  0.0000  0.0000 ]
[ 0.0000  0.0000  0.0000  0.0000 ]
[ 0.5000  0.0000  0.0000  0.5000 ]

Fidelity: 0.8327


## Depolarizing Noise

Apply a single-qubit depolarizing channel on qubit 1 of a Bell pair using `DEPOLARIZE1`.


In [20]:
sc = make_bell()
sc.depolarize(1, 0.3)  # 5% depolarization on qubit 1

print("Circuit:")
print(sc.circuit)

psi = sc.state_vector()
print("\nStatevector after depolarization:")
format_complex(psi)

rho = sc.tomography_dm([0, 1], shots=2000)
print("\nReconstructed rho:")
format_complex(rho)

rho_ideal = np.outer(psi, psi.conj())
print("\nIdeal rho:")
format_complex(rho_ideal)

fidelity = np.real(psi.conj() @ (rho @ psi))
print("\nFidelity:", f"{fidelity:.4f}")


Circuit:
H 0
CX 0 1
DEPOLARIZE1(0.3) 1

Statevector after depolarization:
[ 0.7071  0.0000  0.0000  -0.7071 ]

Reconstructed ρ:
[ 0.3947  -0.0018+0.0110j  -0.0093-0.0067j  0.3010+0.0003j ]
[ -0.0018-0.0110j  0.1013  0.0005+0.0023j  0.0152-0.0053j ]
[ -0.0093+0.0067j  0.0005-0.0023j  0.1067  -0.0083+0.0120j ]
[ 0.3010-0.0003j  0.0152+0.0053j  -0.0083-0.0120j  0.3972 ]

Ideal ρ:
[ 0.5000  0.0000  0.0000  -0.5000 ]
[ 0.0000  0.0000  0.0000  0.0000 ]
[ 0.0000  0.0000  0.0000  0.0000 ]
[ -0.5000  0.0000  0.0000  0.5000 ]

Fidelity: 0.0950


## Idle-Induced Depolarization

Model idling noise on qubit 0 over a duration using either T₂ or T₁. Here we use a pure‐dephasing T₂ formula.


In [21]:
sc = make_bell()
sc.idle(0, time=5.0, T2=30.0)  # idle for 5 units with T2=30

print("Circuit:")
print(sc.circuit)

psi = sc.state_vector()
print("\nStatevector after idling noise:")
format_complex(psi)

rho = sc.tomography_dm([0, 1], shots=2000)
print("\nReconstructed rho:")
format_complex(rho)

rho_ideal = np.outer(psi, psi.conj())
print("\nIdeal rho:")
format_complex(rho_ideal)

fidelity = np.real(psi.conj() @ (rho @ psi))
print("\nFidelity:", f"{fidelity:.4f}")


Circuit:
H 0
CX 0 1
DEPOLARIZE1(0.204691) 0

Statevector after idling noise:
[ 0.7071  0.0000  0.0000  0.7071 ]

Reconstructed ρ:
[ 0.4225  -0.0130+0.0073j  -0.0088+0.0060j  0.3605+0.0073j ]
[ -0.0130-0.0073j  0.0720  -0.0025-0.0097j  -0.0002-0.0015j ]
[ -0.0088-0.0060j  -0.0025+0.0097j  0.0770  -0.0150-0.0203j ]
[ 0.3605-0.0073j  -0.0002+0.0015j  -0.0150+0.0203j  0.4285 ]

Ideal ρ:
[ 0.5000  0.0000  0.0000  0.5000 ]
[ 0.0000  0.0000  0.0000  0.0000 ]
[ 0.0000  0.0000  0.0000  0.0000 ]
[ 0.5000  0.0000  0.0000  0.5000 ]

Fidelity: 0.7860
